In [76]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
import pandas as pd
import numpy as np
import rule_optimisation.optimisation_functions as opt_funcs
import cProfile, pstats

In [78]:
from rule_application.argo_rule_applier import ArgoRuleApplier

In [79]:
om = pd.read_pickle('~/Downloads/omnyex_processed_for_profiling.pkl')
X = om.drop('chargeback', axis=1)
y = om['chargeback']

In [80]:
X = X.astype(float)

In [81]:
dummy_rules = dict((i, f"X['{col}']>=1") for i, col in enumerate(X.columns))

# Old version

In [86]:
filename = 'argo_rule_applier_profile_old.dat'
ara = ArgoRuleApplier(rule_strings=dummy_rules)
cProfile.run('ara.apply(X, y)', sort='cumtime', filename=f'{filename}.dat')

In [87]:
p = pstats.Stats(f'{filename}.dat')
p.sort_stats('cumtime').print_stats()

Fri Jan  8 18:42:18 2021    argo_rule_applier_profile_old.dat.dat

         57323 function calls (57034 primitive calls) in 3.403 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.458    3.458 {built-in method builtins.exec}
        1    0.197    0.197    3.404    3.404 /Users/jlaidler/Documents/tigress/tigress/argo/argo/rule_application/rule_application/argo_rule_applier.py:30(apply)
        1    0.115    0.115    1.757    1.757 /Users/jlaidler/Documents/tigress/tigress/argo/argo/rule_application/rule_application/argo_rule_applier.py:58(_get_rule_descriptions)
        1    0.506    0.506    1.218    1.218 /Users/jlaidler/Documents/tigress/tigress/argo/argo/argo_utils/argo_utils/argo_utils.py:141(return_binary_pred_perf_of_set_numpy)
       53    0.001    0.000    1.186    0.022 /Users/jlaidler/venvs/argo/lib/python3.7/site-packages/pandas/core/internals/managers.py:366(apply)
        1    0

# Old version (with numpy rules)

In [9]:
from rules.rules import Rules

In [10]:
r = Rules(rule_strings=dummy_rules)

In [11]:
dummy_rule_dicts = r.as_rule_dicts()
dummy_rules_np = r.as_rule_strings(as_numpy=True)

In [12]:
filename = 'argo_rule_applier_profile_old_np_rules.dat'
ara = ArgoRuleApplier(rule_strings=dummy_rules_np)
cProfile.run('ara.apply(X, y)', sort='cumtime', filename=f'{filename}.dat')

In [13]:
p = pstats.Stats(f'{filename}.dat')
p.sort_stats('cumtime').print_stats()

Fri Jan  8 18:12:00 2021    argo_rule_applier_profile_old_np_rules.dat.dat

         24084 function calls (23632 primitive calls) in 3.285 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.404    3.404 {built-in method builtins.exec}
        1    0.175    0.175    3.361    3.361 /Users/jlaidler/Documents/tigress/tigress/argo/argo/rule_application/rule_application/argo_rule_applier.py:30(apply)
        1    0.091    0.091    1.651    1.651 /Users/jlaidler/Documents/tigress/tigress/argo/argo/rule_application/rule_application/argo_rule_applier.py:58(_get_rule_descriptions)
        1    0.429    0.429    1.152    1.152 /Users/jlaidler/Documents/tigress/tigress/argo/argo/argo_utils/argo_utils/argo_utils.py:141(return_binary_pred_perf_of_set_numpy)
       57    1.136    0.020    1.136    0.020 {method 'copy' of 'numpy.ndarray' objects}
        1    0.002    0.002    1.131    1.131 /Users/jlaidler/

# Old version (with numpy rules manually changed to .values)

In [9]:
from rules.rules import Rules

In [42]:
dummy_rules_np_man = dict((k, v.replace(".to_numpy(na_value=np.nan)", ".values")) for k, v in dummy_rules_np.items())

In [43]:
filename = 'argo_rule_applier_profile_old_np_rules_changed_values.dat'
ara = ArgoRuleApplier(rule_strings=dummy_rules_np_man)
cProfile.run('ara.apply(X, y)', sort='cumtime', filename=f'{filename}.dat')

In [44]:
p = pstats.Stats(f'{filename}.dat')
p.sort_stats('cumtime').print_stats()

Fri Jan  8 18:24:56 2021    argo_rule_applier_profile_old_np_rules_changed_values.dat.dat

         10234 function calls (10132 primitive calls) in 3.392 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.516    3.516 {built-in method builtins.exec}
        1    0.192    0.192    3.461    3.461 /Users/jlaidler/Documents/tigress/tigress/argo/argo/rule_application/rule_application/argo_rule_applier.py:30(apply)
        1    0.102    0.102    1.779    1.779 /Users/jlaidler/Documents/tigress/tigress/argo/argo/rule_application/rule_application/argo_rule_applier.py:58(_get_rule_descriptions)
        1    0.550    0.550    1.274    1.274 /Users/jlaidler/Documents/tigress/tigress/argo/argo/argo_utils/argo_utils/argo_utils.py:141(return_binary_pred_perf_of_set_numpy)
        1    0.003    0.003    1.101    1.101 /Users/jlaidler/Documents/tigress/tigress/argo/argo/rule_application/rule_application/argo

In [57]:
%timeit [eval(rule_string) for name, rule_string in dummy_rules.items()]

102 ms ± 734 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [58]:
%%timeit 
test = []
for name, rule_string in dummy_rules.items():
    test.append(eval(rule_string))

101 ms ± 1.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
eval_np = np.vectorize(lambda x: eval(x))

In [27]:
dummy_rules_np_arr = np.array(list(dummy_rules_np.values()))

In [36]:
%timeit (X['order_currency_code_AED'].values>=1.0)

973 µs ± 15.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [37]:
%timeit (X['order_currency_code_AED']>=1.0)

1.34 ms ± 505 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [71]:
X_test = pd.Series(np.random.uniform(0.0, 10.0, 10000000))

In [72]:
%timeit X_test>1

7.48 ms ± 1.77 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [73]:
%timeit X_test.values>1

6.15 ms ± 125 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [74]:
%timeit X_test.to_numpy(na_value=np.nan)>1

33 ms ± 253 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [57]:
X_rules_arr = [np.zeros((100000)), np.zeros((100000))]

In [59]:
%timeit pd.DataFrame(X_rules_arr)

3.8 s ± 88.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
%timeit pd.DataFrame(X_rules_arr).T

3.91 s ± 107 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [66]:
%timeit pd.DataFrame(np.asarray([np.zeros((100000)), np.zeros((100000))])).T

6.99 ms ± 226 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [81]:
test_rules = {
        'Rule1': "X['num_distinct_txn_per_email_7day']>=7",
        'Rule2': "X['email_alpharatio']<=0.5",
        'Rule3': "X['num_distinct_txn_per_email_1day']>=1",
        'Rule4': "X['email_kb_distance']<=0.5",
        'Rule5': "X['ip_country_us']==False",
        'Rule6': "X['num_distinct_txn_per_email_1day']<=3",
        'Rule7': "X['num_distinct_txn_per_email_7day']<=5",
        'Rule8': "X['email_kb_distance']>=0.61",
        'Rule9': "X['email_alpharatio']>=0.5"
    }

In [82]:
test_r = Rules(rule_strings=test_rules)

In [83]:
test_r.as_rule_dicts()

{'Rule1': {'condition': 'AND',
  'rules': [{'field': 'num_distinct_txn_per_email_7day',
    'operator': 'greater_or_equal',
    'value': 7.0}]},
 'Rule2': {'condition': 'AND',
  'rules': [{'field': 'email_alpharatio',
    'operator': 'less_or_equal',
    'value': 0.5}]},
 'Rule3': {'condition': 'AND',
  'rules': [{'field': 'num_distinct_txn_per_email_1day',
    'operator': 'greater_or_equal',
    'value': 1.0}]},
 'Rule4': {'condition': 'AND',
  'rules': [{'field': 'email_kb_distance',
    'operator': 'less_or_equal',
    'value': 0.5}]},
 'Rule5': {'condition': 'AND',
  'rules': [{'field': 'ip_country_us', 'operator': 'equal', 'value': False}]},
 'Rule6': {'condition': 'AND',
  'rules': [{'field': 'num_distinct_txn_per_email_1day',
    'operator': 'less_or_equal',
    'value': 3.0}]},
 'Rule7': {'condition': 'AND',
  'rules': [{'field': 'num_distinct_txn_per_email_7day',
    'operator': 'less_or_equal',
    'value': 5.0}]},
 'Rule8': {'condition': 'AND',
  'rules': [{'field': 'email_k

In [84]:
test_r.as_rule_strings(as_numpy=True)

{'Rule1': "(X['num_distinct_txn_per_email_7day'].to_numpy(na_value=np.nan)>=7.0)",
 'Rule2': "(X['email_alpharatio'].to_numpy(na_value=np.nan)<=0.5)",
 'Rule3': "(X['num_distinct_txn_per_email_1day'].to_numpy(na_value=np.nan)>=1.0)",
 'Rule4': "(X['email_kb_distance'].to_numpy(na_value=np.nan)<=0.5)",
 'Rule5': "(X['ip_country_us'].to_numpy(na_value=np.nan)==False)",
 'Rule6': "(X['num_distinct_txn_per_email_1day'].to_numpy(na_value=np.nan)<=3.0)",
 'Rule7': "(X['num_distinct_txn_per_email_7day'].to_numpy(na_value=np.nan)<=5.0)",
 'Rule8': "(X['email_kb_distance'].to_numpy(na_value=np.nan)>=0.61)",
 'Rule9': "(X['email_alpharatio'].to_numpy(na_value=np.nan)>=0.5)"}